[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/telmo-correa/nebulous-llm-experiment/blob/main/notebooks/3%20-%20Agent%20with%20RetrievalQA%20tools.ipynb)

### 3. Agent with RetrievalQA tools

This notebook demonstrates how to create an agent that uses a RetrievalQA chain for each dataset as a tool.

#### Data sources

Unlike in the previous notebook, we have generated separated embeddings for 5 each of the same data sources about Nebulous: Fleet Command:

| Source | Description | Link |
|---|---|---|
| nebfltcom Wiki | Pages from the fan-made wiki | [🔗](http://nebfltcom.wikidot.com/) |
| Steam guides | Contents from some of the most popular Steam guides for Nebulous: Fleet Command focusing on gameplay | [🔗](https://steamcommunity.com/app/887570/guides/) |
| `#new-players` | Text messages from the discord channel `#new-players`, where new players may ask questions | [🔗](https://discord.gg/UT6wU7TQ) |
| `#shipyard` | Text messages from the discord channel `#shipyard`, where players ask for help with builds or fleet/ship-building questions | [🔗](https://discord.gg/UT6wU7TQ) |
| in-game lore | Content from in-game lore | [🔗](https://store.steampowered.com/app/887570/NEBULOUS_Fleet_Command/) |

The five embeddings are provided in a single compressed google drive file: [data_by_source.tar.gz](https://drive.google.com/file/d/1-yDL18Ns0GZZBH8jGKbdMEZHPMwzZqDb)

#### Tools

Rather than having a single chain query a joint data source, we will create 5 different retrieval QA chains, and wrap them into language models that can be used as tools by a main chat agent in order to investigate and answer a user query.

In [ ]:
## If running on Google Colab, install the dependencies:

%pip install openai langchain

In [1]:
import openai
import os

import getpass

from langchain.agents import AgentType, initialize_agent
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.vectorstores import Chroma
from langchain.tools import Tool

Setup OpenAI API key:

In [2]:
if "OPENAI_API_KEY" not in os.environ:
    print("Please enter your OpenAI API key:")
    openai_api_key = getpass.getpass()

    os.environ["OPENAI_API_KEY"] = openai_api_key
    openai.api_key = openai_api_key

Please enter your OpenAI API key:


 ········


Load generated embeddings for each data source:

In [3]:
!wget -O /tmp/data_by_source.tar.gz "<dataset location here>"

Uncompress it:

In [4]:
!tar -xvzf /tmp/data_by_source.tar.gz

data_by_source/
data_by_source/discord_shipyard/
data_by_source/discord_shipyard/chroma-embeddings.parquet.tmp
data_by_source/discord_shipyard/chroma-embeddings.parquet
data_by_source/discord_shipyard/chroma-collections.parquet
data_by_source/discord_shipyard/index/
data_by_source/discord_shipyard/index/uuid_to_id_fe23f4cc-c020-45ae-abe3-7303804586ba.pkl
data_by_source/discord_shipyard/index/id_to_uuid_fe23f4cc-c020-45ae-abe3-7303804586ba.pkl
data_by_source/discord_shipyard/index/index_metadata_fe23f4cc-c020-45ae-abe3-7303804586ba.pkl
data_by_source/discord_shipyard/index/index_fe23f4cc-c020-45ae-abe3-7303804586ba.bin
data_by_source/lore/
data_by_source/lore/chroma-embeddings.parquet
data_by_source/lore/chroma-collections.parquet
data_by_source/lore/index/
data_by_source/lore/index/id_to_uuid_96f26b95-80c9-42fd-b0df-92c3666185fa.pkl
data_by_source/lore/index/index_metadata_96f26b95-80c9-42fd-b0df-92c3666185fa.pkl
data_by_source/lore/index/index_96f26b95-80c9-42fd-b0df-92c3666185fa.bin


Remove downloaded compressed file:

In [5]:
!rm -rf /tmp/data_by_source.tar.gz

Setup embedding locations:

In [6]:
PERSIST_WIKI = "data_by_source/wiki/"
PERSIST_STEAM = "data_by_source/steam/"
PERSIST_DISCORD_NEWPLAYERS = "data_by_source/discord_newplayers/"
PERSIST_DISCORD_SHIPYARD = "data_by_source/discord_shipyard/"
PERSIST_LORE = "data_by_source/lore/"

Setup custom prompt for agents:

In [7]:
prompt_template = """
Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: {context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

Create tools for each data source:

In [8]:
tool_information = [
    {
        "name": "Nebulous wiki",
        "tool_description": """\
This is a large language model with access to contents of a fan-made wiki for Nebulous: Fleet Command.  Input should be a full sentence.
""",
        "persist_directory": PERSIST_WIKI
    },
    {
        "name": "Nebulous Steam guides",
        "tool_description": """\
This is a large language model with access to information from Steam guides for Nebulous: Fleet Command.  Input should be a full sentence.
""",
        "persist_directory": PERSIST_STEAM
    },
    {
        "name": "Nebulous discord #new-players channel",
        "tool_description": """\
This is a large language model with access to information from a discord channel\
in the official server for Nebulous: Fleet Command.  The channel description reads:\
`Questions, answers, and help for new players!`\
Input should be a full sentence.
""",
        "persist_directory": PERSIST_DISCORD_NEWPLAYERS
    },
    {
        "name": "Nebulous discord #shipyard channel",
        "tool_description": """\
This is a large language model with access to information from a discord channel\
in the official server for Nebulous: Fleet Command.  The channel description reads:\
`A place to help or ask for help with builds, ask or answer fleet/ship-building questions.`\
Input should be a full sentence.
""",
        "persist_directory": PERSIST_DISCORD_SHIPYARD
    },
    {
        "name": "Nebulous in-game lore",
        "tool_description": """\
This is a large language model with access to in-game lore for Nebulous: Fleet Command.  Input should be a full sentence.
""",
        "persist_directory": PERSIST_LORE
    },
]

In [9]:
embedding = OpenAIEmbeddings()
llm = OpenAI(temperature=0)

for d in tool_information:
    d["docsearch"] = Chroma(persist_directory=d["persist_directory"], embedding_function=embedding)
    d["chain"] = RetrievalQA.from_chain_type(
        llm,
        chain_type="stuff", 
        retriever=d["docsearch"].as_retriever(search_type="similarity", search_kwargs={"k": 10}),
        chain_type_kwargs=chain_type_kwargs
    )
    d["tool"] = Tool(
        name=d["name"],
        func=d["chain"].run,
        description=d["tool_description"]
    )

In [10]:
tools = [d["tool"] for d in tool_information]

In [11]:
chat = ChatOpenAI(temperature=0)

template="""
You are a helpful AI assistant that can answer questions about Nebulous: Fleet Command.
If the question is not about Nebulous: Fleet Command, do not answer it.

Use the information provided by your tools as context to write your answer.
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [12]:
## Optionally, use W&B to track responses

from langchain.callbacks.tracers import WandbTracer

wandb_config = {"project": "llm_agent_test"}
# tracer = WandbTracer(wandb_config)

In [13]:
llm = ChatOpenAI(temperature=0)

agent = initialize_agent(
    tools, 
    llm, 
    prompt=chat_prompt,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    handle_parsing_errors=True,
    # callbacks=[tracer]
)

In [16]:
agent.run("What are the strategies for playing ANS?")



> Entering new  chain...
I should consult resources to learn about ANS strategies.
Action: Nebulous Steam guides
Action Input: "ANS strategies"
Observation:  ANS strategies include using 120mm or 250mm HE-RPF at a loaded missile carrier, asymmetrical HCs/BBs, jamming incoming missiles, and using AMMs with a blast-fragmentation warhead. OSP strategies include using an Ocello carrying Alliance equipment, using R550 and R400 super-radars, and using comms jammers.
Thought:That's a lot of information to digest. I wonder if there are any more specific tips for ANS gameplay.
Action: Nebulous discord #new-players channel
Action Input: "Any tips for playing ANS?"
Observation:  When playing ANS, make use of their advantages such as electronic warfare, hybrid missile weapons, and particle beams. Utilize their missile advantage by using the "Cruise" upgrade on missiles to waypoint them around rocks and out of LOS. Use floodlights to stay safe from jamming. Try to avoid fights that emphasize your

'ANS strategies include using hybrid missile weapons, particle beams, and electronic warfare to take advantage of their strengths. ANS lore stands for Alliance Naval Ship and is the designation for Alliance warships.'

In [15]:
## If using W & B, make sure tracer is saved

# tracer.finish()